In [ ]:
# This code made by Mohammed Mossad :-)

In [1]:
!pip install datasets
!pip install transformers
!pip install jiwer
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you 

In [ ]:
from huggingface_hub import login
secret_key = ''
login(token = secret_key)

In [3]:
from datasets import load_dataset, DatasetDict

dataset = DatasetDict()
dataset['train'] = load_dataset("mozilla-foundation/common_voice_12_0", "ar", split="train[57%:]", trust_remote_code=True)
dataset['validation'] = load_dataset("mozilla-foundation/common_voice_12_0", "ar", split="validation[90%:]", trust_remote_code=True)
dataset['test'] = load_dataset("mozilla-foundation/common_voice_12_0", "ar", split="test[65%:]", trust_remote_code=True)

# Keep only audio and sentence columns
columns_to_keep = ['audio', 'sentence']
dataset = dataset.remove_columns([col for col in dataset['train'].column_names if col not in columns_to_keep])


README.md:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

common_voice_12_0.py:   0%|          | 0.00/8.08k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.57k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/63.2k [00:00<?, ?B/s]

n_shards.json:   0%|          | 0.00/12.7k [00:00<?, ?B/s]

...


Reading metadata...: 10433it [00:00, 174401.18it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 17531it [00:00, 175287.18it/s]
Reading metadata...: 36219it [00:00, 163565.23it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14974it [00:00, 140276.35it/s]


In [4]:
import re

def filter_empty(example):
    return example["sentence"].strip() != ""

def text_preprocessing(batch):
    batch['sentence'] = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', batch['sentence'])
    batch['sentence'] = re.sub(r'[^\u0621-\u063A\u0641-\u064A\s]', '', batch['sentence'])
    batch['sentence'] = re.sub(r'[إأآا]', 'ا', batch['sentence'])
    batch['sentence'] = re.sub(r'ى', 'ي', batch['sentence'])
    batch['sentence'] = re.sub(r'ؤ', 'و', batch['sentence'])
    batch['sentence'] = re.sub(r'ئ', 'ي', batch['sentence'])
    batch['sentence'] = re.sub(r'ة', 'ه', batch['sentence'])
    batch['sentence'] = re.sub(r'\s+', ' ', batch['sentence']).strip()
    return batch

dataset = dataset.filter(filter_empty)
dataset = dataset.map(text_preprocessing)

Filter:   0%|          | 0/12124 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1035 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3652 [00:00<?, ? examples/s]

Map:   0%|          | 0/12124 [00:00<?, ? examples/s]

Map:   0%|          | 0/1035 [00:00<?, ? examples/s]

Map:   0%|          | 0/3652 [00:00<?, ? examples/s]

In [5]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration


model_pth = '/kaggle/input/whisper-ar/transformers/default/2/whisper-arabic-finetuned-best'
model = WhisperForConditionalGeneration.from_pretrained(model_pth).to("cuda")
processor = WhisperProcessor.from_pretrained(model_pth)

# Print basic model architecture info
print("Model base:", model.base_model_prefix)
print("Model architecture:", model.__class__.__name__)
print("Model size (parameters):", sum(p.numel() for p in model.parameters()) / 1e6, "M")

# Print tokenizer info
print("Tokenizer vocab size:", processor.tokenizer.vocab_size)

2025-05-11 01:53:59.779673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746928439.985033      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746928440.044824      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model base: model
Model architecture: WhisperForConditionalGeneration
Model size (parameters): 241.734912 M
Tokenizer vocab size: 50258


In [6]:
import librosa  # Ensure you have librosa for resampling

def prepare_dataset(batch):
    audio = batch["audio"]
    # Resample audio from 48kHz to 16kHz (Whisper's expected sampling rate)
    audio_resampled = librosa.resample(audio["array"], orig_sr=audio["sampling_rate"], target_sr=16000)
    # Process the resampled audio with Whisper's feature extractor
    inputs = processor(audio_resampled, sampling_rate=16000, return_tensors="pt")
    # Store input features and labels
    batch["input_features"] = inputs.input_features[0]
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids

    return batch

dataset = dataset.map(prepare_dataset, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/12124 [00:00<?, ? examples/s]

Map:   0%|          | 0/1035 [00:00<?, ? examples/s]

Map:   0%|          | 0/3652 [00:00<?, ? examples/s]

In [7]:
import torch
from tqdm import tqdm
import pandas as pd
import evaluate

wer_metric = evaluate.load("wer")
pred_str = []
label_str = []

# Reset generation config to remove forced_decoder_ids if it exists
model.generation_config.forced_decoder_ids = None

for item in tqdm(dataset["train"]):
    input_features = torch.tensor(item["input_features"]).unsqueeze(0).to("cuda")

    with torch.no_grad():
        # Generate with task and language parameters
        generated_ids = model.generate(
            input_features, 
            task="transcribe", 
            language="ar",
            generation_config=model.generation_config  # Use the updated config
        )

    # Decode prediction
    prediction = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Prepare reference
    reference_ids = item["labels"]
    reference_ids = [id if id != -100 else processor.tokenizer.pad_token_id for id in reference_ids]
    reference = processor.tokenizer.decode(reference_ids, skip_special_tokens=True)

    # Check if prediction and reference are non-empty
    if prediction.strip() and reference.strip():
        pred_str.append(prediction)
        label_str.append(reference)

# Compute WERs
wers = [wer_metric.compute(predictions=[p], references=[r]) for p, r in zip(pred_str, label_str)]

# Save to CSV
df_train = pd.DataFrame({"reference": label_str, "prediction": pred_str, "wer": wers})
df_train.to_csv("wer_train.csv", index=False)

  0%|          | 0/12124 [00:00<?, ?it/s]You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
The attention mask is not set and cannot be inferred fr

In [8]:
wers = [wer_metric.compute(predictions=[p], references=[r]) for p, r in zip(pred_str, label_str)]

# Save to CSV
df_train = pd.DataFrame({"reference": label_str, "prediction": pred_str, "wer": wers})
df_train.to_csv("wer_train.csv", index=False)

In [9]:
print("\n--- Final Evaluation on Training Set ---\n")
print('\n TRAINING WER -->' , {df_train['wer'].mean()})


--- Final Evaluation on Training Set ---


 TRAINING WER --> {0.010225739642963102}


In [10]:
df_train.head(10)

,reference,prediction,wer
0,الحمد لله رب العالمين,الحمد لله رب العالمين,0.0
1,ووهبنا لداوود سليمان نعم العبد انه اواب,ووهبنا لداوود سليمان نعم العبد انه اواب,0.0
2,لم يصدق احد قصته,لم يصدق احد قصته,0.0
3,ولا يملك الذين يدعون من دونه الشفاعه الا من شه...,ولا يملك الذين يدعون من دونه الشفاعه الا من شه...,0.0
4,لقوه شهواتهم,لقوه شهواتهم,0.0
5,حميه الكيتو نظام غذاييه تتناول فيه كميه كبيره ...,حميه الكيتو لظام الغذاييه تتناول فيه كميه كبير...,0.2
6,وقال بعض الادباء من عرف معابه فلا يلم من عابه,وقال بعض الادباء من عرف معابه فلا يلم من عابه,0.0
7,ولا دوله لك من اذاله منك,ولا دوله لك من اذاله منك,0.0
8,كل شيء اذا كثر رخص الا العقل فانه اذا كثر غلا,كل شيء اذا كثر رخص الا العقل فانه اذا كثر غلا,0.0
9,حذرتها من المجيء الي هنا,حذرتها من المجيء الي هنا,0.0


In [11]:
import torch
from tqdm import tqdm
import pandas as pd
import evaluate

wer_metric = evaluate.load("wer")
pred_str = []
label_str = []

# Reset generation config to remove forced_decoder_ids if it exists
model.generation_config.forced_decoder_ids = None

for item in tqdm(dataset["validation"]):
    input_features = torch.tensor(item["input_features"]).unsqueeze(0).to("cuda")

    with torch.no_grad():
        # Generate with task and language parameters
        generated_ids = model.generate(
            input_features, 
            task="transcribe", 
            language="ar",
            generation_config=model.generation_config  # Use the updated config
        )

    # Decode prediction
    prediction = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Prepare reference
    reference_ids = item["labels"]
    reference_ids = [id if id != -100 else processor.tokenizer.pad_token_id for id in reference_ids]
    reference = processor.tokenizer.decode(reference_ids, skip_special_tokens=True)

    pred_str.append(prediction)
    label_str.append(reference)

# Compute WERs
wers = [wer_metric.compute(predictions=[p], references=[r]) for p, r in zip(pred_str, label_str)]

# Save to CSV
df_valid = pd.DataFrame({"reference": label_str, "prediction": pred_str, "wer": wers})
df_valid.to_csv("wer_valid.csv", index=False)

100%|██████████| 1035/1035 [08:11<00:00,  2.10it/s]


In [12]:
print("\n--- Final Evaluation on Validation Set ---\n")
print('\n VALIDATION WER -->' , {df_valid['wer'].mean()})


--- Final Evaluation on Validation Set ---


 VALIDATION WER --> {0.35761631283370415}


In [13]:
df_valid.head(20)

,reference,prediction,wer
0,فهي سلطان قاهر تتالف من رهبته الاهواء المختلفه,فهي سلطان قاهر تتعلم من رهبته الاهواء المختلفه,0.125000
1,وربما كان فيه منيته كاخذ الادويه التي فيها شفا...,ورد ما كان فيه منيته كاخذ الادويه التي في عشرا...,0.500000
2,جانبي الايسر قلبه الفواد وبلادي هي لي قلبي اليمين,جانبي الايسر قلبه الفواد وابلاد يهاني قلب ينين,0.555556
3,ضحك الجميع علي,رحكي الجميعانيه,1.000000
4,من صمم هذا المتحف,نممسن نهارا المدحفه,1.000000
5,اعوذ بالله من الشيطان الرجيم,اعوذ بالله من الشيطان الرلييم,0.200000
6,وقالوا ما لنا لا نري رجالا كنا نعدهم من الاشرار,وقالوا ما لنا لنري رجالا كنا نعجهم من الاشرار,0.300000
7,كم من الوقت ستمضي هنا,كم من الوقت ستنظي هنا,0.200000
8,اسرع كي تلحق بالقطار,ارسر كيد الحق بالقهار,1.000000
9,وسبب المواصله بينهما وجود الاتفاق منهما فصارت ...,والسبب المواصله بينهما وجود الانفاق منهما فصار...,0.500000


In [14]:
import torch
from tqdm import tqdm
import pandas as pd
import evaluate

wer_metric = evaluate.load("wer")
pred_str = []
label_str = []

# Reset generation config to remove forced_decoder_ids if it exists
model.generation_config.forced_decoder_ids = None

for item in tqdm(dataset["test"]):
    input_features = torch.tensor(item["input_features"]).unsqueeze(0).to("cuda")

    with torch.no_grad():
        # Generate with task and language parameters
        generated_ids = model.generate(
            input_features, 
            task="transcribe", 
            language="ar",
            generation_config=model.generation_config  # Use the updated config
        )

    # Decode prediction
    prediction = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Prepare reference
    reference_ids = item["labels"]
    reference_ids = [id if id != -100 else processor.tokenizer.pad_token_id for id in reference_ids]
    reference = processor.tokenizer.decode(reference_ids, skip_special_tokens=True)

    pred_str.append(prediction)
    label_str.append(reference)

# Compute WERs
wers = [wer_metric.compute(predictions=[p], references=[r]) for p, r in zip(pred_str, label_str)]

# Save to CSV
df_test = pd.DataFrame({"reference": label_str, "prediction": pred_str, "wer": wers})
df_test.to_csv("wer_test.csv", index=False)

100%|██████████| 3652/3652 [27:04<00:00,  2.25it/s]


In [15]:
print("\n--- Final Evaluation on Test Set ---\n")
print('\n TEST WER -->' , {df_test['wer'].mean()})


--- Final Evaluation on Test Set ---


 TEST WER --> {0.3790301573502559}


In [16]:
df_test.head(20)

,reference,prediction,wer
0,هل يمكنني استعاره سيارتك لهذه الليله,هل يمكنني استياره سيارات كري عليه الليله,0.666667
1,هل هذه المعلومه صحيحه,الهذه المعلومه صحيصه,0.750000
2,كنت مشغولا البارحه,كنت مشغولا البارح,0.333333
3,كم طولك,كانتولك,1.000000
4,سيصبح ابني معلما,سيصبح ابنه عليما,0.666667
5,ذاك المنزل صغير جدا,ذاك المنزل صغير جدا,0.000000
6,علينا ان نسقي الازهار,ارينا ان نصقل الزرام,0.750000
7,نحن مسرورون,نحن مسترورون,0.500000
8,اقوم بالتمارين الرياضيه لاحافظ علي صحتي,اكم بالدمار عند رياضيه لاحافظ علي صحه,0.833333
9,انه شخص اناني للغايه,اناء شخص اناني للغايه,0.250000


In [ ]:
# Code by Mohammed Mossad 